# Inserting the necessary packages

In [2]:
import os
import matplotlib.pyplot as plt      
import numpy as np
import PIL                      # to import image
import tensorflow as tf
import keras                    # to import resnet50 model
from keras.layers import Dense,Flatten     # to add additional layers to resnet layer
from keras.models import Sequential         # to sequentially add layers to model
from tensorflow.keras.optimizers import Adam    # use adam optimizer while compiling
from flask import Flask, request, redirect, render_template
from werkzeug.utils import secure_filename


# Loading the saved model and creating object for Flask Connectivity

In [3]:
from PIL import Image
import base64
import matplotlib.pyplot as plt
from io import BytesIO
import matplotlib.pyplot as mtp
allowed_exts = {'jpg', 'jpeg','png','JPG','JPEG','PNG'}

reconstructed_model = keras.models.load_model("my_model_glucoma")
app = Flask(__name__)

def check_allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in allowed_exts

# Loading the training dataset

In [4]:
import pickle
pickle_in = open("train_ds","rb")
classnames = pickle.load(pickle_in)

# accepting the image retina file and predicting the output using model

In [5]:

import cv2
@app.route("/",methods=['GET', 'POST'])
def index():
	if request.method == 'POST':
		if 'file' not in request.files:
			print('No file attached in request')
			return redirect(request.url)
		file = request.files['file']
		if file.filename == '':
			print('No file selected')
			return redirect(request.url)
		if file and check_allowed_file(file.filename):
			filename = secure_filename(file.filename)
			print(filename)
			img = Image.open(file.stream)
			img_height,img_width=300,300
			image_path = "C:/Users/Sinegalatha/Desktop/Imarticus Learning/GLAUCOMA/static/Images/"+filename
			print(image_path)
			mtp.imshow(img)
			mtp.show() 
			img1=np.asarray(img)
			cv2.imwrite(image_path, img1)
			bad=cv2.imread(image_path)
			bad_resized= cv2.resize(bad, (img_height,img_width))
			bad=np.expand_dims(bad_resized,axis=0)
			pred_save=reconstructed_model.predict(bad)
			print(pred_save)
			output_class=classnames[np.argmax(pred_save)]
			print("The predicted class is", output_class)
			
			with BytesIO() as buf:
				img.save(buf, 'jpeg')
				image_bytes = buf.getvalue()
			encoded_string = base64.b64encode(image_bytes).decode()         
		return render_template('index.html', img_data=encoded_string, answer=output_class), 200
	else:
		return render_template('index.html', img_data=""), 200


In [ ]:
if __name__ == "__main__":
	app.debug=True
	app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jul/2022 22:51:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jul/2022 22:51:17] "GET /favicon.ico HTTP/1.1" 404 -
